In [3]:
#!pip -q install -U openai==1.81.0 langgraph langchain-upstage langchain-community chromadb transformers python-dotenv pydantic rich


In [4]:
#!pip install chromadb

In [5]:
#!pip -q install -U trafilatura readability-lxml beautifulsoup4 lxml

In [30]:
import os, json, time, math, re, hashlib, textwrap
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, TypedDict, Literal

import requests
from dotenv import load_dotenv
from pydantic import BaseModel, Field, ValidationError

from openai import OpenAI
from transformers import AutoTokenizer

import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings, PersistentClient

from rich import print as rprint


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## 1) API Keys & Clients

- `UPSTAGE_API_KEY`, `SERPER_API_KEY`는 **이미 등록되어 있고 변수명도 동일**하다고 했으니 그대로 씁니다.
- 모델 라인업(예시):  
  - Solar: `solar-pro2-250909`  
  - Document Parse: `document-parse-250618`  
  - Embedding: `solar-embedding-1-large-query`


In [34]:
load_dotenv()

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
SERPER_API_KEY  = os.getenv("SERPER_API_KEY")
DART_API_KEY    = os.getenv("DART_API_KEY")  # 선택(없어도 동작)

# Colab userdata (선택)
try:
    from google.colab import userdata
    UPSTAGE_API_KEY = UPSTAGE_API_KEY or userdata.get("UPSTAGE_API_KEY")
    SERPER_API_KEY  = SERPER_API_KEY  or userdata.get("SERPER_API_KEY")
    DART_API_KEY    = DART_API_KEY    or userdata.get("DART_API_KEY")
except Exception:
    pass

assert UPSTAGE_API_KEY, "UPSTAGE_API_KEY not found"
assert SERPER_API_KEY, "SERPER_API_KEY not found"

client = OpenAI(base_url="https://api.upstage.ai/v1", api_key=UPSTAGE_API_KEY)

# 토큰 추정용 (HF 토크나이저는 model_max_length=4096 경고가 뜨는 경우가 많아서 무력화)
tokenizer = AutoTokenizer.from_pretrained("upstage/solar-pro-preview-instruct")
tokenizer.model_max_length = 1_000_000

# Solar-Pro2 컨텍스트(≈64K) 안전값
MAX_CONTEXT_LIMIT = 65000

# Vector DB
CHROMA_PATH = "./chroma_db_ideaproof"
chroma_client = PersistentClient(path=CHROMA_PATH)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

In [35]:
import inspect

def function_to_schema(func) -> dict:
    sig = inspect.signature(func)
    props = {}
    required = []
    for name, param in sig.parameters.items():
        if name in ("self",):
            continue
        ann = param.annotation
        jtype = "string"
        if ann in (int,):
            jtype = "integer"
        elif ann in (float,):
            jtype = "number"
        elif ann in (bool,):
            jtype = "boolean"
        elif ann in (list, List):
            jtype = "array"
        elif ann in (dict, Dict):
            jtype = "object"
        props[name] = {"type": jtype}
        if param.default is inspect._empty:
            required.append(name)
    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {"type": "object", "properties": props, "required": required}
        }
    }

def truncate_tokens_if_needed(tokenizer, agent_instructions, messages, content, max_token_limit=None):
    """
    - base가 이미 limit을 넘으면(히스토리 과다) 에러 내지 말고 내용을 최소화해서 계속 진행
    - content가 넘치면 content만 잘라서 limit 안으로 넣기
    """
    if max_token_limit is None:
        max_token_limit = MAX_CONTEXT_LIMIT

    inputs = tokenizer.apply_chat_template(
        [{"role": "system", "content": agent_instructions}] + messages,
        tokenize=True
    )
    base_tokens = len(inputs)

    if base_tokens >= max_token_limit:
        return "[...omitted due to context budget...]"

    enc = tokenizer.encode(content)
    if base_tokens + len(enc) > max_token_limit:
        keep = max_token_limit - base_tokens
        enc = enc[:max(0, keep)]
        content = tokenizer.decode(enc, skip_special_tokens=True) + "\n\n[...Content Truncated due to Context Limit...]"
    return content

def execute_tool_call(tool_name: str, tools: Dict[str, Any], args: Dict[str, Any]) -> str:
    if tool_name not in tools:
        raise KeyError(f"Tool not found: {tool_name}")
    return tools[tool_name](**args)

def safe_json_loads(s: str) -> Any:
    s = s.strip()
    try:
        return json.loads(s)
    except Exception:
        pass
    s2 = re.sub(r"^```(json)?\s*|\s*```$", "", s, flags=re.MULTILINE).strip()
    try:
        return json.loads(s2)
    except Exception:
        pass
    m = re.search(r"(\{.*\}|\[.*\])", s2, flags=re.DOTALL)
    if not m:
        raise ValueError("No JSON object found in text")
    return json.loads(m.group(1))

def hash_key(*parts: str) -> str:
    h = hashlib.sha256()
    for p in parts:
        h.update(p.encode("utf-8"))
    return h.hexdigest()[:16]


## 2) Shared Utils (schema, tool runner, token budget)

- Upstage Solar-Pro 계열은 컨텍스트가 대략 64K 수준이므로, 안전한 한계치로 60K를 사용합니다.


In [36]:
import inspect

def function_to_schema(func) -> dict:
    sig = inspect.signature(func)
    props = {}
    required = []
    for name, param in sig.parameters.items():
        if name in ("self",):
            continue
        ann = param.annotation
        jtype = "string"
        if ann in (int,):
            jtype = "integer"
        elif ann in (float,):
            jtype = "number"
        elif ann in (bool,):
            jtype = "boolean"
        elif ann in (list, List):
            jtype = "array"
        elif ann in (dict, Dict):
            jtype = "object"
        props[name] = {"type": jtype}
        if param.default is inspect._empty:
            required.append(name)
    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {"type": "object", "properties": props, "required": required}
        }
    }

def truncate_tokens_if_needed(tokenizer, agent_instructions, messages, content, max_token_limit=None):
    """
    - base가 이미 limit을 넘으면(히스토리 과다) 에러 내지 말고 내용을 최소화해서 계속 진행
    - content가 넘치면 content만 잘라서 limit 안으로 넣기
    """
    if max_token_limit is None:
        max_token_limit = MAX_CONTEXT_LIMIT

    inputs = tokenizer.apply_chat_template(
        [{"role": "system", "content": agent_instructions}] + messages,
        tokenize=True
    )
    base_tokens = len(inputs)

    if base_tokens >= max_token_limit:
        return "[...omitted due to context budget...]"

    enc = tokenizer.encode(content)
    if base_tokens + len(enc) > max_token_limit:
        keep = max_token_limit - base_tokens
        enc = enc[:max(0, keep)]
        content = tokenizer.decode(enc, skip_special_tokens=True) + "\n\n[...Content Truncated due to Context Limit...]"
    return content

def execute_tool_call(tool_name: str, tools: Dict[str, Any], args: Dict[str, Any]) -> str:
    if tool_name not in tools:
        raise KeyError(f"Tool not found: {tool_name}")
    return tools[tool_name](**args)

def safe_json_loads(s: str) -> Any:
    s = s.strip()
    try:
        return json.loads(s)
    except Exception:
        pass
    s2 = re.sub(r"^```(json)?\s*|\s*```$", "", s, flags=re.MULTILINE).strip()
    try:
        return json.loads(s2)
    except Exception:
        pass
    m = re.search(r"(\{.*\}|\[.*\])", s2, flags=re.DOTALL)
    if not m:
        raise ValueError("No JSON object found in text")
    return json.loads(m.group(1))

def hash_key(*parts: str) -> str:
    h = hashlib.sha256()
    for p in parts:
        h.update(p.encode("utf-8"))
    return h.hexdigest()[:16]


## 3) Core Tools

필수 Tool 기능:
1) 인터넷 검색(serper.dev)  
2) 인터넷 파일 다운로드  
3) PDF → Markdown 파싱(Upstage Document Parse)  
4) Vector DB 저장/조회(Chroma + Upstage embedding)  
5) LLM-as-Judge Rerank (Top-K 재정렬)

> 한국 기업/시장 분석을 우선하기 위해 검색 쿼리에 한국 소스 힌트를 자동으로 섞습니다.


In [37]:
class Agent(BaseModel):
    name: str = "Agent"
    model: str = "solar-pro2-250909"
    instructions: str = "You are a helpful agent."
    tools: List[Any] = Field(default_factory=list)

def run_agent(messages: List[Dict[str, Any]], agent: Agent, max_context_limit: int = None) -> str:
    """
    OpenAI tool-calling 스타일 루프 실행.
    - tools가 없으면 tools/tool_choice 파라미터를 아예 보내지 않는다(Upstage 400 방지)
    """
    if max_context_limit is None:
        max_context_limit = MAX_CONTEXT_LIMIT

    tool_schemas = [function_to_schema(t) for t in agent.tools]
    tool_map = {t.__name__: t for t in agent.tools}

    while True:
        kwargs = dict(
            model=agent.model,
            messages=[{"role":"system","content": agent.instructions}] + messages,
        )
        if tool_schemas:
            kwargs["tools"] = tool_schemas
            kwargs["tool_choice"] = "auto"

        resp = client.chat.completions.create(**kwargs)
        msg = resp.choices[0].message

        if not getattr(msg, "tool_calls", None):
            content = msg.content or ""
            content = truncate_tokens_if_needed(tokenizer, agent.instructions, messages, content, max_token_limit=max_context_limit)
            return content

        for tc in msg.tool_calls:
            tool_name = tc.function.name
            args = json.loads(tc.function.arguments or "{}")
            try:
                out = execute_tool_call(tool_name, tool_map, args)
                if not isinstance(out, str):
                    out = json.dumps(out, ensure_ascii=False)
            except Exception as e:
                out = f"ToolError: {e}"

            out = truncate_tokens_if_needed(tokenizer, agent.instructions, messages, out, max_token_limit=max_context_limit)

            messages.append({"role":"assistant","content": None, "tool_calls":[tc]})
            messages.append({"role":"tool","tool_call_id": tc.id, "content": out})


In [38]:
class RequestModel(BaseModel):
    raw_request: str
    language: str = "ko"
    tone: str = "concise"
    mode: Literal["fast","standard","deep"] = "standard"

class IdeaSchemaModel(BaseModel):
    problem: str
    target: str
    solution: str
    differentiation: str
    business_model: str
    industry: str
    keywords: List[str] = Field(default_factory=list)
    persona_hypotheses: List[str] = Field(default_factory=list)

class EvidenceQueryPlan(BaseModel):
    question: str
    queries: List[str]
    preferred_sources: List[str] = Field(default_factory=list)

class EvidenceItem(BaseModel):
    source_url: str
    title: str = ""
    snippet: str = ""
    local_path: Optional[str] = None
    parsed_markdown: Optional[str] = None

class EvidenceStoreModel(BaseModel):
    collection: str
    items: List[EvidenceItem] = Field(default_factory=list)
    version: str = "v1"

class SignalsModel(BaseModel):
    market: str
    competition: str
    customer: str
    risks: str
    score_explainable: Dict[str, float]

class VerdictModel(BaseModel):
    decision: Literal["GO","NO_GO","PIVOT"]
    key_reasons: List[str]
    evidence_links: List[str]
    next_actions: List[str]

class ArtifactsModel(BaseModel):
    prd_1p: str
    scope_must_should_could: str
    erd_mermaid: str
    user_flow: str
    roadmap_2_4_weeks: str
    validation_plan: str

class GuardsModel(BaseModel):
    policy_violation: bool = False
    token_overflow: bool = False
    copyright_risk: bool = False
    evidence_insufficient: bool = False
    notes: List[str] = Field(default_factory=list)


class MarketModelModel(BaseModel):
    # C 파트(거시→미시→생태계) 요약
    market_scope: str
    tam_sam_som: Dict[str, Any] = Field(default_factory=dict)
    cagr: Dict[str, Any] = Field(default_factory=dict)
    ecosystem: Dict[str, Any] = Field(default_factory=dict)
    risks: List[str] = Field(default_factory=list)
    evidence_links: List[str] = Field(default_factory=list)
    confidence: Dict[str, float] = Field(default_factory=dict)

class CostSimModel(BaseModel):
    # B 파트(최소비용 개발계획 + 비용/일정 시뮬레이션)
    mvp_scope: Dict[str, Any] = Field(default_factory=dict)
    wbs: Dict[str, Any] = Field(default_factory=dict)
    validation_plan: Dict[str, Any] = Field(default_factory=dict)
    cost_model_assumptions: Dict[str, Any] = Field(default_factory=dict)
    simulation: Dict[str, Any] = Field(default_factory=dict)
    min_to_validate: Dict[str, Any] = Field(default_factory=dict)
    assumptions_and_unknowns: List[Dict[str, Any]] = Field(default_factory=list)

class WorkflowState(TypedDict, total=False):
    request: Dict[str, Any]
    idea_schema: Dict[str, Any]
    evidence_plan: List[Dict[str, Any]]
    evidence_store: Dict[str, Any]
    evidence_pack: List[Dict[str, Any]]
    market_plan: Dict[str, Any]
    market_model: Dict[str, Any]
    cost_sim: Dict[str, Any]
    signals: Dict[str, Any]
    verdict: Dict[str, Any]
    artifacts: Dict[str, Any]
    guards: Dict[str, Any]
    final_report_markdown: str
    intake: Dict[str, Any]
    logs: List[Dict[str, Any]]


In [39]:
import logging
import time
import trafilatura
from urllib.parse import urlparse

logging.getLogger("trafilatura").setLevel(logging.ERROR)
logging.getLogger("trafilatura.core").setLevel(logging.ERROR)
logging.getLogger("trafilatura.utils").setLevel(logging.ERROR)

def web_search(query: str, k: int = 10) -> List[Dict[str, str]]:
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query, "num": k})
    headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}
    r = requests.post(url, headers=headers, data=payload, timeout=60)
    r.raise_for_status()
    data = r.json()
    out = []
    for item in data.get("organic", [])[:k]:
        out.append({"title": item.get("title",""), "link": item.get("link",""), "snippet": item.get("snippet","")})
    return out

def is_pdf_url(url: str) -> bool:
    return url.lower().split("?")[0].endswith(".pdf")

def fetch_url_text(url: str, timeout: int = 30, max_chars: int = 30000) -> str:
    try:
        r = requests.get(
            url,
            timeout=timeout,
            headers={"User-Agent":"Mozilla/5.0"},
            allow_redirects=True,
        )
        r.raise_for_status()

        ctype = (r.headers.get("content-type") or "").lower()
        if ("text/html" not in ctype) and ("application/xhtml" not in ctype):
            return ""

        html = (r.text or "").strip()
        if len(html) < 200:
            return ""

        text = trafilatura.extract(html, include_comments=False, include_tables=False, favor_recall=True) or ""
        text = text.strip()
        if len(text) > max_chars:
            text = text[:max_chars]
        return text
    except Exception:
        return ""

def source_priority_score(url: str) -> int:
    host = (urlparse(url).netloc or "").lower()
    score = 0
    if host.endswith(".go.kr"): score += 50
    if host.endswith(".ac.kr"): score += 35
    if host.endswith(".or.kr"): score += 25
    if host.endswith(".re.kr"): score += 20
    if host.endswith(".kr"): score += 10
    if "kosis" in host or "kostat" in host: score += 50
    if "dart" in host or "fss" in host: score += 40
    if "nipa" in host or "kisdi" in host or "kised" in host: score += 25
    return score

def download_file(url: str, save_dir: str = "./downloads") -> str:
    os.makedirs(save_dir, exist_ok=True)
    fn = re.sub(r"[^a-zA-Z0-9_.-]", "_", url.split("/")[-1]) or f"file_{int(time.time())}"
    path = os.path.join(save_dir, fn)
    r = requests.get(url, timeout=120, headers={"User-Agent":"Mozilla/5.0"}, allow_redirects=True)
    r.raise_for_status()
    with open(path, "wb") as f:
        f.write(r.content)
    return path

def parse_pdf_to_markdown(pdf_path: str) -> str:
    url = "https://api.upstage.ai/v1/document-ai/document-parse"
    headers = {"Authorization": f"Bearer {UPSTAGE_API_KEY}"}
    with open(pdf_path, "rb") as f:
        files = {"document": f}
        data = {
            "model": "document-parse-250618",
            "ocr": "auto",
            "chart_recognition": True,
            "coordinates": True,
            "output_formats": '["markdown"]',
            "base64_encoding": '["figure"]',
        }
        r = requests.post(url, headers=headers, files=files, data=data, timeout=180)
        r.raise_for_status()
        j = r.json()
    return j.get("content", {}).get("markdown", "")

def _normalize_for_embedding(x: Any, max_chars: int = 8000) -> str:
    s = (x if isinstance(x, str) else str(x) if x is not None else "").replace("\x00", "").strip()
    if not s:
        return ""
    if len(s) > max_chars:
        s = s[:max_chars]
    return s

class UpstageEmbeddingFunction(EmbeddingFunction):
    def __init__(
        self,
        client: OpenAI,
        model: str = "solar-embedding-1-large-query",
        batch_size: int = 16,
        max_chars: int = 8000,
        retries: int = 1,
        backoff_sec: float = 1.0,
    ):
        self.client = client
        self.model = model
        self.batch_size = batch_size
        self.max_chars = max_chars
        self.retries = retries
        self.backoff_sec = backoff_sec

    def _embed_batch(self, texts: List[str]) -> List[List[float]]:
        last_err = None
        for attempt in range(self.retries + 1):
            try:
                resp = self.client.embeddings.create(model=self.model, input=texts)
                return [d.embedding for d in resp.data]
            except Exception as e:
                last_err = e
                if attempt < self.retries:
                    time.sleep(self.backoff_sec * (attempt + 1))
        raise last_err

    def _embed_one(self, text: str) -> List[float]:
        resp = self.client.embeddings.create(model=self.model, input=text)
        return resp.data[0].embedding

    def __call__(self, input: Documents) -> Embeddings:
        if isinstance(input, str):
            t = _normalize_for_embedding(input, max_chars=self.max_chars)
            return [self._embed_one(t)] if t else [self._embed_one(".")]

        raw = [_normalize_for_embedding(t, max_chars=self.max_chars) for t in list(input)]
        texts = [t if t else "." for t in raw]

        out: List[List[float]] = []
        i = 0
        while i < len(texts):
            batch = texts[i:i+self.batch_size]
            try:
                out.extend(self._embed_batch(batch))
            except Exception:
                for t in batch:
                    out.append(self._embed_one(t if t else "."))
            i += self.batch_size
        return out

embedding_fn = UpstageEmbeddingFunction(client)

def get_collection(name: str):
    return chroma_client.get_or_create_collection(name=name, embedding_function=embedding_fn)

def vectordb_upsert(collection: str, docs: List[str], metadatas: List[Dict[str, Any]], ids: List[str]) -> int:
    clean_docs, clean_metas, clean_ids = [], [], []
    for d, m, i in zip(docs, metadatas, ids):
        s = _normalize_for_embedding(d, max_chars=8000)
        if not s:
            continue
        clean_docs.append(s)
        clean_metas.append(m)
        clean_ids.append(i)

    if not clean_ids:
        return 0

    col = get_collection(collection)
    col.upsert(documents=clean_docs, metadatas=clean_metas, ids=clean_ids)
    return len(clean_ids)

def vectordb_query(collection: str, query: str, n_results: int = 8) -> Dict[str, Any]:
    col = get_collection(collection)
    return col.query(query_texts=[query], n_results=n_results)

def _extract_rerank_items(data: Any) -> List[Dict[str, Any]]:
    """Robustly extract rerank score items from various LLM JSON shapes.

    Acceptable shapes seen in the wild:
    - {"scores": [ {i, score, reason}, ... ]}
    - [{"i":..., "score":...}, ...]
    - [{"results": [...] }] / {"results":[...]}
    - {"ranking": {"1": {...}, "2": {...}, ...}} or {"ranking":[...]}
    - Nested wrappers containing any of the above
    """
    def _as_list(x: Any) -> List[Any]:
        return x if isinstance(x, list) else []

    # If it's already a list, it might be either the items OR a wrapper list containing a dict.
    if isinstance(data, list):
        flattened: List[Dict[str, Any]] = []
        for el in data:
            flattened.extend(_extract_rerank_items(el))
        # If we managed to extract anything from elements, use that.
        if flattened:
            return flattened
        # Otherwise assume it's already the item list.
        return [el for el in data if isinstance(el, dict)]

    if isinstance(data, dict):
        # direct hits
        if isinstance(data.get("scores"), list):
            return [el for el in data["scores"] if isinstance(el, dict)]
        if isinstance(data.get("results"), list):
            return [el for el in data["results"] if isinstance(el, dict)]
        if "ranking" in data:
            r = data["ranking"]
            if isinstance(r, list):
                return [el for el in r if isinstance(el, dict)]
            if isinstance(r, dict):
                # Often ranking keys are "1","2",...
                try:
                    keys = sorted(r.keys(), key=lambda k: int(str(k)))
                except Exception:
                    keys = list(r.keys())
                return [r[k] for k in keys if isinstance(r.get(k), dict)]

        # recursive search through nested values (first match wins)
        for v in data.values():
            items = _extract_rerank_items(v)
            if items:
                return items

    return []

def llm_rerank(query: str, candidates: List[Dict[str, Any]], top_k: int = 5) -> List[Dict[str, Any]]:
    packed = [{"i": i, "text": (c.get("text","")[:1200]), "meta": c.get("meta",{})} for i,c in enumerate(candidates)]
    prompt = {
        "task": "rerank",
        "schema": {"scores": [{"i": 0, "score": 0.0, "reason": "string"}]},
        "instruction": (
            "Score EACH candidate 0-10 by relevance to the query. "
            "Prefer authoritative Korea-specific sources. "
            "Return JSON ONLY exactly in this schema: "
            "{\"scores\": [{\"i\": 0, \"score\": 8.5, \"reason\": \"...\"}, ...]}"
        ),
        "query": query,
        "candidates": packed
    }

    resp = client.chat.completions.create(
        model="solar-pro2-250909",
        messages=[
            {"role": "system", "content": "Return JSON only. No prose. No markdown."},
            {"role": "user", "content": json.dumps(prompt, ensure_ascii=False)}
        ],
    )

    raw = resp.choices[0].message.content
    data = safe_json_loads(raw)

    scored_items = _extract_rerank_items(data)

    norm: List[Tuple[int, float, str]] = []
    for item in scored_items:
        if not isinstance(item, dict):
            print(f"Warning: Item in scored list is not a dictionary. Skipping: {item}")
            continue

        idx = item.get("i", item.get("index", item.get("id", item.get("idx"))))
        if idx is None:
            print(f"Warning: Item in scored list missing 'i' key: {item}. Skipping.")
            continue
        try:
            idx = int(idx)
        except Exception:
            print(f"Warning: Item index 'i' is not an integer: {item}. Skipping.")
            continue
        if idx < 0 or idx >= len(candidates):
            print(f"Warning: Item index {idx} out of bounds (candidates len {len(candidates)}): {item}. Skipping.")
            continue

        score = item.get("score", item.get("sco", item.get("s", item.get("relevance", 0))))
        try:
            score_f = float(score)
        except Exception:
            print(f"Warning: Item score is not a float: {item}. Defaulting to 0.0.")
            score_f = 0.0

        reason = item.get("reason") or item.get("rationale") or item.get("why") or ""
        norm.append((idx, score_f, str(reason)))

    # Keep best score per candidate index
    best: Dict[int, Tuple[float, str]] = {}
    for idx, score_f, reason in norm:
        if (idx not in best) or (score_f > best[idx][0]):
            best[idx] = (score_f, reason)

    if not best:
        # Fallback: prioritize by source domain quality if rerank output is unusable
        merged = []
        for c in candidates:
            meta = c.get("meta", {}) or {}
            url = meta.get("url") or meta.get("link") or ""
            merged.append({**c, "score": float(source_priority_score(url)), "reason": "fallback: source_priority_score"})
        merged.sort(key=lambda x: x["score"], reverse=True)
        return merged[:top_k]

    merged = [{**candidates[i], "score": best[i][0], "reason": best[i][1]} for i in best]
    merged.sort(key=lambda x: x["score"], reverse=True)
    return merged[:top_k]

def simple_chunk(text: str, max_chars: int = 1500, overlap: int = 200) -> List[str]:
    text = (text or "").strip()
    if not text:
        return []
    chunks = []
    i = 0
    step = max(1, max_chars - overlap)
    while i < len(text):
        c = text[i:i+max_chars].strip()
        if c:
            chunks.append(c)
        i += step
    return chunks
EVIDENCE_PLAN_PROMPT = '''
역할: 아이디어를 검증하기 위한 '질문 리스트'와 각 질문별 '검색 쿼리'를 만든다.
출력(JSON only):
[
  {"question":"...", "queries":["...","...","..."], "preferred_sources":["gov","kosis","dart","research"]},
  ...
]
규칙:
- 한국 시장/한국 기업 중심 쿼리로 작성
- 각 question당 queries는 3개 이내
- question은 4~6개
'''
evidence_plan_agent = Agent(
    name="EvidencePlanMaker",
    instructions=EVIDENCE_PLAN_PROMPT,
    tools=[],
)

def make_evidence_plan(idea_schema: Dict[str, Any]) -> List[Dict[str, Any]]:
    messages = [{"role":"user","content": json.dumps({"idea_schema": idea_schema}, ensure_ascii=False)}]
    out = run_agent(messages, evidence_plan_agent)
    plan = safe_json_loads(out)
    if not isinstance(plan, list):
        raise ValueError("evidence_plan must be a list")
    return plan

def check_vectordb_cache(collection: str) -> bool:
    try:
        chroma_client.get_collection(name=collection)
        return True
    except Exception:
        return False

def build_expanded_queries(q: str) -> List[str]:
    years = ["2025", "2024", "2023"]
    tails = ["시장 규모", "시장 동향 보고서", "통계", "백서", "TAM SAM SOM", "경쟁사", "규제", "지원사업"]
    out = []
    for y in years:
        out.append(f"{q} {y}")
    for t in tails:
        out.append(f"{q} {t}")
    out += [f"{q} site:go.kr", f"{q} site:kosis.kr", f"{q} DART 공시"]
    return list(dict.fromkeys(out))

def evidence_builder_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    idea = state["idea_schema"]
    collection = f"ideaproof_{hash_key(json.dumps(idea, ensure_ascii=False))}"

    mode = (state.get("request", {}) or {}).get("mode", "standard")
    if mode == "fast":
        MAX_RESULTS_PER_QUERY_1 = 5
        MAX_SOURCES_1 = 12
        MAX_CHUNKS_PER_SOURCE = 6
        MAX_TOTAL_CHUNKS = 120
        MIN_STORED_CHUNKS = 50
        DO_EXPAND = False
        DO_PDF_PARSE = False
        TIME_BUDGET_SEC = 6 * 60
    elif mode == "deep":
        MAX_RESULTS_PER_QUERY_1 = 10
        MAX_SOURCES_1 = 45
        MAX_CHUNKS_PER_SOURCE = 14
        MAX_TOTAL_CHUNKS = 450
        MIN_STORED_CHUNKS = 180
        DO_EXPAND = True
        DO_PDF_PARSE = True
        TIME_BUDGET_SEC = 18 * 60
    else:
        MAX_RESULTS_PER_QUERY_1 = 8
        MAX_SOURCES_1 = 25
        MAX_CHUNKS_PER_SOURCE = 10
        MAX_TOTAL_CHUNKS = 260
        MIN_STORED_CHUNKS = 120
        DO_EXPAND = True
        DO_PDF_PARSE = True
        TIME_BUDGET_SEC = 12 * 60

    def time_left() -> float:
        return TIME_BUDGET_SEC - (time.time() - t0)

    if check_vectordb_cache(collection):
        state["evidence_store"] = EvidenceStoreModel(collection=collection, items=[], version="v3").model_dump()
        logs.append({"node":"evidence_builder", "t": time.time()-t0, "cache":"HIT", "collection": collection, "mode": mode})
        state["logs"] = logs
        return state

    plan = make_evidence_plan(idea)
    state["evidence_plan"] = plan

    def run_harvest(queries: List[str], max_results_per_query: int, max_sources_total: int):
        results = []
        for q in queries:
            if time_left() <= 0:
                break
            try:
                for r in web_search(q, k=max_results_per_query):
                    if r.get("link"):
                        results.append(r)
            except Exception:
                continue

        uniq = {}
        for r in results:
            uniq[r["link"]] = r
        ranked = list(uniq.values())
        ranked.sort(key=lambda x: source_priority_score(x["link"]), reverse=True)
        ranked = ranked[:max_sources_total]

        items: List[EvidenceItem] = []
        all_chunks, all_metas, all_ids = [], [], []

        for r in ranked:
            if time_left() <= 0 or len(all_chunks) >= MAX_TOTAL_CHUNKS:
                break

            url = r["link"]
            title = r.get("title","")
            snippet = r.get("snippet","")

            text = ""
            local = None
            md_text = None

            if is_pdf_url(url):
                if DO_PDF_PARSE and source_priority_score(url) >= 30 and time_left() > 60:
                    try:
                        local = download_file(url)
                        md_text = parse_pdf_to_markdown(local)
                        text = (md_text or "").strip()
                    except Exception:
                        text = ""
                else:
                    text = ""
            else:
                text = fetch_url_text(url)

            if not text:
                text = f"{title}\n{snippet}\nURL: {url}"

            items.append(EvidenceItem(source_url=url, title=title, snippet=snippet, local_path=local, parsed_markdown=md_text))

            chunks = simple_chunk(text, max_chars=1500, overlap=200)[:MAX_CHUNKS_PER_SOURCE]
            for j, ch in enumerate(chunks):
                if len(all_chunks) >= MAX_TOTAL_CHUNKS:
                    break
                cid = hash_key(collection, url, str(j))
                all_chunks.append(ch)
                all_metas.append({"url": url, "title": title, "chunk": j})
                all_ids.append(cid)

        stored = 0
        if all_chunks:
            stored = vectordb_upsert(collection, all_chunks, all_metas, all_ids)
        return items, stored

    base_queries = []
    for p in plan:
        for q in (p.get("queries") or [])[:3]:
            base_queries.append(q)
    base_queries = list(dict.fromkeys(base_queries))[:15]

    items, stored = run_harvest(base_queries, max_results_per_query=MAX_RESULTS_PER_QUERY_1, max_sources_total=MAX_SOURCES_1)

    if DO_EXPAND and (stored < MIN_STORED_CHUNKS) and (time_left() > 90):
        expanded = []
        for p in plan:
            expanded += build_expanded_queries(p.get("question",""))
        expanded = list(dict.fromkeys(expanded))[:20]

        items2, stored2 = run_harvest(expanded, max_results_per_query=MAX_RESULTS_PER_QUERY_1, max_sources_total=max(MAX_SOURCES_1, 30))
        merged = {it.source_url: it for it in (items + items2)}
        items = list(merged.values())
        stored = max(stored, stored2)

    state["evidence_store"] = EvidenceStoreModel(collection=collection, items=items, version="v3").model_dump()

    logs.append({
        "node":"evidence_builder",
        "t": time.time()-t0,
        "cache":"MISS",
        "collection": collection,
        "mode": mode,
        "sources": len(items),
        "stored_chunks": stored,
        "time_budget_sec": TIME_BUDGET_SEC,
        "time_budget_hit": (time_left() <= 0)
    })
    state["logs"] = logs
    return state


In [40]:
INTAKE_PROMPT = """
역할: 요청을 '아이디어 검증/설계 워크플로'로 처리할지, 일상대화로 처리할지 라우팅한다.
목표:
1) request 정규화(언어/톤/모드)
2) 아이디어 입력이 부족하면 '최소 질문'으로 보완 질문을 만든다.
출력(JSON only):
{
  "route": "workflow" | "chat",
  "request": {"raw_request": "...", "language": "ko", "tone": "concise", "mode":"fast|standard|deep"},
  "missing_fields": ["problem","target","solution","differentiation","business_model"],
  "clarifying_questions": ["...","..."]
}
규칙:
- 질문은 최대 5개. 선택형/단답형 우선.
"""

intake_agent = Agent(name="IntakeRouteClarify", instructions=INTAKE_PROMPT, tools=[])

def intake_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    raw = state["request"]["raw_request"]
    messages = [{"role":"user","content": raw}]
    out = run_agent(messages, intake_agent)
    data = safe_json_loads(out)

    state["request"] = data["request"]
    state["intake"] = data

    guards = state.get("guards", {})
    guards.setdefault("notes", [])
    guards["notes"].append(f"route={data.get('route')}")
    state["guards"] = guards

    logs.append({"node":"intake", "t": time.time()-t0, "route": data.get("route")})
    state["logs"] = logs
    return state

def route_after_intake(state: WorkflowState) -> str:
    route = state.get("intake", {}).get("route", "workflow")
    return "chat_end" if route == "chat" else "structurer"


In [41]:
STRUCTURER_PROMPT = """
역할: 아이디어를 문제/대상/해결/차별/BM로 구조화하고, 산업 분류 및 키워드를 만든다.
입력:
- raw idea text(자유형)
출력(JSON only):
{
  "problem": "...",
  "target": "...",
  "solution": "...",
  "differentiation": "...",
  "business_model": "...",
  "industry": "...",
  "keywords": ["..."],
  "persona_hypotheses": ["..."]
}
규칙:
- 모호하면 가능한 가설을 1~2개로 제한해 persona_hypotheses에 넣고, 단정하지 말 것.
"""

structurer_agent = Agent(name="Structurer", instructions=STRUCTURER_PROMPT, tools=[])

def structurer_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    raw = state["request"]["raw_request"]
    messages = [{"role":"user","content": raw}]
    out = run_agent(messages, structurer_agent)
    data = safe_json_loads(out)

    obj = IdeaSchemaModel(**data)
    state["idea_schema"] = obj.model_dump()

    logs.append({"node":"structurer", "t": time.time()-t0, "industry": obj.industry})
    state["logs"] = logs
    return state


In [42]:
EVIDENCE_PLAN_PROMPT = '''
역할: 아이디어를 검증하기 위한 '질문 리스트'와 각 질문별 '검색 쿼리'를 만든다.
출력(JSON only):
[
  {"question":"...", "queries":["...","...","..."], "preferred_sources":["gov","kosis","dart","research"]},
  ...
]
규칙:
- 한국 시장/한국 기업 중심 쿼리로 작성
- 각 question당 queries는 3개 이내
- question은 4~6개
'''
evidence_plan_agent = Agent(
    name="EvidencePlanMaker",
    instructions=EVIDENCE_PLAN_PROMPT,
    tools=[],
)

def make_evidence_plan(idea_schema: Dict[str, Any]) -> List[Dict[str, Any]]:
    messages = [{"role":"user","content": json.dumps({"idea_schema": idea_schema}, ensure_ascii=False)}]
    out = run_agent(messages, evidence_plan_agent)
    plan = safe_json_loads(out)
    if not isinstance(plan, list):
        raise ValueError("evidence_plan must be a list")
    return plan


MARKET_PLAN_PROMPT_KR_V1 = """역할: 한국(대한민국) 창업 아이디어를 검증하기 위한 '거시→자산군→업종/세그먼트→생태계' 리서치 플랜을 만든다.
출력(JSON only):
{
  "market_definition": {
    "one_line_definition": "...",
    "industry_labels": {"ksic_candidates":["..."], "industry_keywords_kr":["..."]},
    "segment_definition": {"who":"...", "where":"대한민국", "use_case":"...", "pricing_anchor":"..."},
    "value_chain_position": "..."
  },
  "layers": [
    {
      "layer_name": "macro|asset_class|industry_segment|ecosystem_firms",
      "goals": ["..."],
      "metric_targets": [{"name":"...", "why_it_matters":"...", "preferred_source_hint":"..."}],
      "query_sets": [{"perspective":"시장|규제|고객|기술|재무|리스크|경쟁", "preferred_sources":["..."], "queries":["...","..."]}],
      "min_evidence": 3,
      "caps": {"max_queries":12, "max_sources":20, "max_per_domain":2},
      "success_criteria": ["..."],
      "fallback": {"if_metrics_missing":["..."], "notes":"..."}
    }
  ]
}
규칙:
- 범위는 '대한민국'으로 고정. 해외/글로벌 언급 금지.
- 숫자는 '원하는 지표'로만 제시(근거 없으면 생성 금지). 대신 success_criteria에 '정량 2개 이상 확보' 같은 기준을 둬라.
- caps는 반드시 지켜라.
- ksic_candidates는 확신 없으면 후보 1~3개로만."""

market_plan_agent = Agent(
    name="MarketPlanKR",
    instructions=MARKET_PLAN_PROMPT_KR_V1,
    tools=[],
)

def make_market_plan(idea_schema: Dict[str, Any], raw_request: str = "") -> Dict[str, Any]:
    payload = {"idea_schema": idea_schema, "raw_request": raw_request}
    messages = [{"role":"user","content": json.dumps(payload, ensure_ascii=False)}]
    out = run_agent(messages, market_plan_agent)
    plan = safe_json_loads(out)
    if not isinstance(plan, dict):
        raise ValueError("market_plan must be a dict")
    return plan

def flatten_market_plan_to_evidence_plan(market_plan: Dict[str, Any], max_questions: int = 6) -> List[Dict[str, Any]]:
    # market_plan.layers[*].query_sets 를 evidence_builder가 쓰는 (question, queries, preferred_sources) 리스트로 평탄화
    layers = (market_plan or {}).get("layers", []) or []
    out: List[Dict[str, Any]] = []
    for layer in layers:
        layer_name = (layer.get("layer_name","") or "").strip() or "layer"
        goals = layer.get("goals", []) or []
        goal = goals[0] if goals else "핵심 근거 수집"
        query_sets = layer.get("query_sets", []) or []
        queries: List[str] = []
        preferred_sources: List[str] = []
        for qs in query_sets:
            preferred_sources += (qs.get("preferred_sources", []) or [])
            for q in (qs.get("queries", []) or []):
                if isinstance(q, str) and q.strip():
                    queries.append(q.strip())
        # caps: question당 3개 이내
        queries = list(dict.fromkeys(queries))[:3]
        preferred_sources = list(dict.fromkeys([s for s in preferred_sources if isinstance(s, str) and s.strip()]))[:6]

        out.append({
            "question": f"[{layer_name}] {goal}",
            "queries": queries if queries else [f"{layer_name} 대한민국 시장 규모 통계", f"{layer_name} 규제 동향 대한민국"],
            "preferred_sources": preferred_sources,
        })
        if len(out) >= max_questions:
            break

    # 최소 4개는 유지(부족하면 기본 질문으로 보강)
    while len(out) < 4:
        out.append({
            "question": "[general] 대한민국 시장/경쟁/규제 정량 근거 보강",
            "queries": ["대한민국 시장 규모 통계", "대한민국 경쟁사 리스트", "대한민국 관련 규제 요약"],
            "preferred_sources": ["kosis","dart","gov"],
        })
    return out[:max_questions]


def check_vectordb_cache(collection: str) -> bool:
    try:
        chroma_client.get_collection(name=collection)
        return True
    except Exception:
        return False

def build_expanded_queries(q: str) -> List[str]:
    years = ["2025", "2024", "2023"]
    tails = ["시장 규모", "시장 동향 보고서", "통계", "백서", "TAM SAM SOM", "경쟁사", "규제", "지원사업"]
    out = []
    for y in years:
        out.append(f"{q} {y}")
    for t in tails:
        out.append(f"{q} {t}")
    out += [f"{q} site:go.kr", f"{q} site:kosis.kr", f"{q} DART 공시"]
    return list(dict.fromkeys(out))

def market_plan_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    idea = state.get("idea_schema", {}) or {}
    raw = (state.get("request", {}) or {}).get("raw_request","")

    try:
        mp = make_market_plan(idea_schema=idea, raw_request=raw)
    except Exception as e:
        mp = {"error": str(e), "layers": []}

    state["market_plan"] = mp

    if not state.get("evidence_plan"):
        try:
            state["evidence_plan"] = flatten_market_plan_to_evidence_plan(mp, max_questions=6)
        except Exception:
            state["evidence_plan"] = make_evidence_plan(idea) if idea else []

    logs.append({"node":"market_plan", "t": time.time()-t0, "layers": len((mp or {}).get("layers", []) or [])})
    state["logs"] = logs
    return state

def evidence_builder_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    idea = state["idea_schema"]
    collection = f"ideaproof_{hash_key(json.dumps(idea, ensure_ascii=False))}"

    mode = (state.get("request", {}) or {}).get("mode", "standard")
    if mode == "fast":
        MAX_RESULTS_PER_QUERY_1 = 5
        MAX_SOURCES_1 = 12
        MAX_CHUNKS_PER_SOURCE = 6
        MAX_TOTAL_CHUNKS = 120
        MIN_STORED_CHUNKS = 50
        DO_EXPAND = False
        DO_PDF_PARSE = False
        TIME_BUDGET_SEC = 6 * 60
    elif mode == "deep":
        MAX_RESULTS_PER_QUERY_1 = 10
        MAX_SOURCES_1 = 45
        MAX_CHUNKS_PER_SOURCE = 14
        MAX_TOTAL_CHUNKS = 450
        MIN_STORED_CHUNKS = 180
        DO_EXPAND = True
        DO_PDF_PARSE = True
        TIME_BUDGET_SEC = 18 * 60
    else:
        MAX_RESULTS_PER_QUERY_1 = 8
        MAX_SOURCES_1 = 25
        MAX_CHUNKS_PER_SOURCE = 10
        MAX_TOTAL_CHUNKS = 260
        MIN_STORED_CHUNKS = 120
        DO_EXPAND = True
        DO_PDF_PARSE = True
        TIME_BUDGET_SEC = 12 * 60

    def time_left() -> float:
        return TIME_BUDGET_SEC - (time.time() - t0)

    if check_vectordb_cache(collection):
        state["evidence_store"] = EvidenceStoreModel(collection=collection, items=[], version="v3").model_dump()
        logs.append({"node":"evidence_builder", "t": time.time()-t0, "cache":"HIT", "collection": collection, "mode": mode})
        state["logs"] = logs
        return state

    plan = state.get("evidence_plan")
    if not plan:
        mp = state.get("market_plan")
        if not mp:
            mp = make_market_plan(idea_schema=idea, raw_request=(state.get("request", {}) or {}).get("raw_request",""))
            state["market_plan"] = mp
        try:
            plan = flatten_market_plan_to_evidence_plan(mp, max_questions=6)
        except Exception:
            plan = make_evidence_plan(idea)
    state["evidence_plan"] = plan

    def run_harvest(queries: List[str], max_results_per_query: int, max_sources_total: int):
        results = []
        for q in queries:
            if time_left() <= 0:
                break
            try:
                for r in web_search(q, k=max_results_per_query):
                    if r.get("link"):
                        results.append(r)
            except Exception:
                continue

        uniq = {}
        for r in results:
            uniq[r["link"]] = r
        ranked = list(uniq.values())
        ranked.sort(key=lambda x: source_priority_score(x["link"]), reverse=True)
        ranked = ranked[:max_sources_total]

        items: List[EvidenceItem] = []
        all_chunks, all_metas, all_ids = [], [], []

        for r in ranked:
            if time_left() <= 0 or len(all_chunks) >= MAX_TOTAL_CHUNKS:
                break

            url = r["link"]
            title = r.get("title","")
            snippet = r.get("snippet","")

            text = ""
            local = None
            md_text = None

            if is_pdf_url(url):
                if DO_PDF_PARSE and source_priority_score(url) >= 30 and time_left() > 60:
                    try:
                        local = download_file(url)
                        md_text = parse_pdf_to_markdown(local)
                        text = (md_text or "").strip()
                    except Exception:
                        text = ""
                else:
                    text = ""
            else:
                text = fetch_url_text(url)

            if not text:
                text = f"{title}\n{snippet}\nURL: {url}"

            items.append(EvidenceItem(source_url=url, title=title, snippet=snippet, local_path=local, parsed_markdown=md_text))

            chunks = simple_chunk(text, max_chars=1500, overlap=200)[:MAX_CHUNKS_PER_SOURCE]
            for j, ch in enumerate(chunks):
                if len(all_chunks) >= MAX_TOTAL_CHUNKS:
                    break
                cid = hash_key(collection, url, str(j))
                all_chunks.append(ch)
                all_metas.append({"url": url, "title": title, "chunk": j})
                all_ids.append(cid)

        stored = 0
        if all_chunks:
            stored = vectordb_upsert(collection, all_chunks, all_metas, all_ids)
        return items, stored

    base_queries = []
    for p in plan:
        for q in (p.get("queries") or [])[:3]:
            base_queries.append(q)
    base_queries = list(dict.fromkeys(base_queries))[:15]

    items, stored = run_harvest(base_queries, max_results_per_query=MAX_RESULTS_PER_QUERY_1, max_sources_total=MAX_SOURCES_1)

    if DO_EXPAND and (stored < MIN_STORED_CHUNKS) and (time_left() > 90):
        expanded = []
        for p in plan:
            expanded += build_expanded_queries(p.get("question",""))
        expanded = list(dict.fromkeys(expanded))[:20]

        items2, stored2 = run_harvest(expanded, max_results_per_query=MAX_RESULTS_PER_QUERY_1, max_sources_total=max(MAX_SOURCES_1, 30))
        merged = {it.source_url: it for it in (items + items2)}
        items = list(merged.values())
        stored = max(stored, stored2)

    state["evidence_store"] = EvidenceStoreModel(collection=collection, items=items, version="v3").model_dump()

    logs.append({
        "node":"evidence_builder",
        "t": time.time()-t0,
        "cache":"MISS",
        "collection": collection,
        "mode": mode,
        "sources": len(items),
        "stored_chunks": stored,
        "time_budget_sec": TIME_BUDGET_SEC,
        "time_budget_hit": (time_left() <= 0)
    })
    state["logs"] = logs
    return state


In [43]:
def extractor_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    plan = state.get("evidence_plan", [])
    collection = state["evidence_store"]["collection"]

    evidence_pack = []
    for p in plan[:6]:
        q = p.get("question","")
        if not q:
            continue

        raw = vectordb_query(collection=collection, query=q, n_results=12)
        docs = raw.get("documents", [[]])[0]
        metas = raw.get("metadatas", [[]])[0]

        if not docs:
            continue

        candidates = [{"text": d, "meta": m} for d, m in zip(docs, metas)]
        reranked = llm_rerank(query=q, candidates=candidates, top_k=5)

        for r in reranked:
            r["text"] = (r.get("text","")[:800]).strip()

        evidence_pack.append({"question": q, "top_chunks": reranked})

    state["evidence_pack"] = evidence_pack

    guards = state.get("guards", {})
    if not evidence_pack:
        guards["evidence_insufficient"] = True
        guards.setdefault("notes", []).append("Extractor: evidence_pack is empty.")
    state["guards"] = guards

    logs.append({"node":"extractor", "t": time.time()-t0, "questions": len(evidence_pack)})
    state["logs"] = logs
    return state


In [44]:
import numpy as np

MARKET_MODEL_PROMPT_KR_V1 = """역할: (대한민국 한정) evidence_pack과 market_plan을 이용해 Scale-up(C) 분석을 만든다.
출력(JSON only):
{
  "market_scope":"...",
  "tam_sam_som": {
    "tam_krw_range":{"min":0,"base":0,"max":0,"basis":"EVIDENCE|ASSUMPTION","notes":"..."},
    "sam_krw_range":{"min":0,"base":0,"max":0,"basis":"EVIDENCE|ASSUMPTION","notes":"..."},
    "som_krw_range":{"min":0,"base":0,"max":0,"basis":"EVIDENCE|ASSUMPTION","notes":"..."}
  },
  "cagr":{
    "scenario_pct":{"conservative":0.0,"base":0.0,"aggressive":0.0,"basis":"EVIDENCE|ASSUMPTION"},
    "how_estimated":"..."
  },
  "ecosystem":{
    "taxonomy":["..."],
    "firms":[{"name":"...","category":"...","size_hint":"대|중|소|스타트업|공공","evidence_url":""}],
    "distribution_summary":"...",
    "concentration_risk":"..."
  },
  "risks":["..."],
  "evidence_links":["..."],
  "confidence":{"market_size":0.0,"cagr":0.0,"ecosystem":0.0}
}
규칙:
- 범위는 대한민국. 해외/글로벌 데이터 금지.
- evidence_links는 evidence_pack.meta.url에서만 선택(최소 3개).
- 숫자는 evidence_pack에 근거가 없으면 0으로 두고 basis를 ASSUMPTION으로 표시 + notes/how_estimated에 검증 방법을 써라.
- firms는 최대 25개. 중복 제거."""
market_model_agent = Agent(
    name="MarketModelKR",
    instructions=MARKET_MODEL_PROMPT_KR_V1,
    tools=[],
)

def market_model_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    payload = {
        "idea_schema": state.get("idea_schema", {}),
        "market_plan": state.get("market_plan", {}),
        "evidence_pack": state.get("evidence_pack", []),
    }
    messages = [{"role":"user","content": json.dumps(payload, ensure_ascii=False)}]
    out = run_agent(messages, market_model_agent)
    data = safe_json_loads(out)

    obj = MarketModelModel(**data)
    state["market_model"] = obj.model_dump()

    logs.append({"node":"market_model", "t": time.time()-t0})
    state["logs"] = logs
    return state


COST_PLAN_PROMPT_KR_V1 = """역할: 아이디어만으로 '최소비용 MVP 개발 계획(B)'을 설계하고, 시뮬레이션 가능한 입력(3점 추정/비용 범위)을 만든다.
출력(JSON only):
{
  "mvp_scope":{"must":[...],"should":[...],"could":[...]},
  "wbs":{
    "milestones":[
      {"id":"M1","name":"...","exit_criteria":["..."],"is_validation_gate":false},
      {"id":"M2","name":"...","exit_criteria":["..."],"is_validation_gate":true}
    ],
    "work_packages":[
      {"id":"WP1","milestone_id":"M1","name":"...","deliverable":"...","stage":"foundation|mvp|validation|nice_to_have",
       "dependencies":["WP0"],"roles":["PM","BE","FE","ML","DS","DESIGN"],
       "effort_days_pert":{"optimistic":1.0,"most_likely":2.0,"pessimistic":4.0},
       "risk_notes":["..."]
      }
    ]
  },
  "validation_plan":{
    "hypotheses":[{"id":"H1","statement":"...","risk_if_false":"..."}],
    "experiments":[{"id":"E1","goal":"...","method":"...","metric":"...","success_threshold":"...","min_sample":"...","milestone_id":"M2"}]
  },
  "cost_model_assumptions":{
    "role_day_rate_krw_range":{
      "PM":{"min":0,"base":0,"max":0},
      "BE":{"min":0,"base":0,"max":0},
      "FE":{"min":0,"base":0,"max":0},
      "ML":{"min":0,"base":0,"max":0},
      "DS":{"min":0,"base":0,"max":0},
      "DESIGN":{"min":0,"base":0,"max":0}
    },
    "infra_monthly_krw_range":{"min":0,"base":0,"max":0},
    "llm_api_monthly_krw_range":{"min":0,"base":0,"max":0},
    "data_purchase_oneoff_krw_range":{"min":0,"base":0,"max":0},
    "overhead_pct_range":{"min":0.0,"base":0.0,"max":0.0}
  },
  "assumptions_and_unknowns":[{"type":"ASSUMPTION","statement":"...","impact":"...","how_to_verify":"..."}]
}
규칙:
- 대한민국 창업/개발 관점에서 현실적인 숫자 범위를 제시하되, 전부 '가정'임을 assumptions_and_unknowns에 명시해라.
- work_packages는 12~25개 수준으로. dependencies는 DAG(순환 금지).
- is_validation_gate=true 인 milestone은 1개만(검증 완료 게이트).
- stage는 반드시 지정.
- effort_days_pert는 person-day 기준."""
cost_plan_agent = Agent(
    name="CostPlanKR",
    instructions=COST_PLAN_PROMPT_KR_V1,
    tools=[],
)

def _sample_pert(a: float, m: float, b: float, size: int, lamb: float = 4.0) -> np.ndarray:
    a, m, b = float(a), float(m), float(b)
    if b <= a:
        return np.full(size, a, dtype=float)
    # beta-PERT
    alpha = 1.0 + lamb * (m - a) / (b - a)
    beta = 1.0 + lamb * (b - m) / (b - a)
    x = np.random.beta(alpha, beta, size=size)
    return a + x * (b - a)

def _sample_tri(min_v: float, mode_v: float, max_v: float, size: int) -> np.ndarray:
    min_v, mode_v, max_v = float(min_v), float(mode_v), float(max_v)
    if max_v <= min_v:
        return np.full(size, min_v, dtype=float)
    mode_v = min(max(mode_v, min_v), max_v)
    return np.random.triangular(min_v, mode_v, max_v, size=size)

def _topo_order(nodes: List[str], deps: Dict[str, List[str]]) -> List[str]:
    # Kahn
    indeg = {n: 0 for n in nodes}
    adj = {n: [] for n in nodes}
    for n in nodes:
        for d in deps.get(n, []) or []:
            if d in indeg:
                adj[d].append(n)
                indeg[n] += 1
    q = [n for n in nodes if indeg[n] == 0]
    out = []
    while q:
        n = q.pop(0)
        out.append(n)
        for nxt in adj.get(n, []):
            indeg[nxt] -= 1
            if indeg[nxt] == 0:
                q.append(nxt)
    if len(out) != len(nodes):
        # cycle fallback: keep given order
        return nodes
    return out

def _simulate_cost_plan(plan: Dict[str, Any], iterations: int = 5000, seed: int = 42) -> Dict[str, Any]:
    np.random.seed(seed)
    wbs = (plan or {}).get("wbs", {}) or {}
    wps = wbs.get("work_packages", []) or []

    # Build WP maps
    wp_ids = []
    deps = {}
    roles_by_wp = {}
    pert_by_wp = {}
    milestone_by_wp = {}
    for wp in wps:
        wid = wp.get("id")
        if not wid:
            continue
        wp_ids.append(wid)
        deps[wid] = wp.get("dependencies", []) or []
        roles_by_wp[wid] = wp.get("roles", []) or []
        e = (wp.get("effort_days_pert", {}) or {})
        pert_by_wp[wid] = (e.get("optimistic", 1.0), e.get("most_likely", 2.0), e.get("pessimistic", 4.0))
        milestone_by_wp[wid] = wp.get("milestone_id","")

    wp_ids = list(dict.fromkeys(wp_ids))
    order = _topo_order(wp_ids, deps)

    # Sample durations
    dur = np.zeros((iterations, len(wp_ids)), dtype=float)
    id_to_idx = {wid:i for i,wid in enumerate(wp_ids)}
    for wid, (a,m,b) in pert_by_wp.items():
        i = id_to_idx.get(wid)
        if i is None:
            continue
        dur[:, i] = _sample_pert(a,m,b,iterations)

    # Schedule (earliest finish)
    ef = np.zeros_like(dur)
    for wid in order:
        i = id_to_idx[wid]
        dep_ids = [d for d in deps.get(wid, []) or [] if d in id_to_idx]
        if not dep_ids:
            ef[:, i] = dur[:, i]
        else:
            dep_idx = [id_to_idx[d] for d in dep_ids]
            ef[:, i] = dur[:, i] + np.max(ef[:, dep_idx], axis=1)
    total_effort_days = np.sum(dur, axis=1)
    project_duration_days = np.max(ef, axis=1)  # dependency-aware

    # Cost
    ass = (plan or {}).get("cost_model_assumptions", {}) or {}
    role_rates = ass.get("role_day_rate_krw_range", {}) or {}
    # sample one rate per role per iteration
    role_rate_samples = {}
    for role, r in role_rates.items():
        if not isinstance(r, dict):
            continue
        role_rate_samples[role] = _sample_tri(r.get("min",0), r.get("base",0), r.get("max",0), iterations)
    # labor cost
    labor = np.zeros(iterations, dtype=float)
    for wid in wp_ids:
        i = id_to_idx[wid]
        roles = roles_by_wp.get(wid, []) or []
        if not roles:
            continue
        rates = []
        for role in roles:
            if role in role_rate_samples:
                rates.append(role_rate_samples[role])
        if not rates:
            continue
        avg_rate = np.mean(np.vstack(rates), axis=0)
        labor += dur[:, i] * avg_rate

    overhead = ass.get("overhead_pct_range", {}) or {}
    overhead_pct = _sample_tri(overhead.get("min",0.0), overhead.get("base",0.0), overhead.get("max",0.0), iterations)
    labor_with_overhead = labor * (1.0 + overhead_pct)

    # recurring
    def _monthly_cost(key: str) -> np.ndarray:
        r = ass.get(key, {}) or {}
        return _sample_tri(r.get("min",0), r.get("base",0), r.get("max",0), iterations)

    months = project_duration_days / 20.0
    infra = months * _monthly_cost("infra_monthly_krw_range")
    llm = months * _monthly_cost("llm_api_monthly_krw_range")

    data_oneoff_r = ass.get("data_purchase_oneoff_krw_range", {}) or {}
    data_oneoff = _sample_tri(data_oneoff_r.get("min",0), data_oneoff_r.get("base",0), data_oneoff_r.get("max",0), iterations)

    total_cost = labor_with_overhead + infra + llm + data_oneoff

    def pct(x, p):
        return float(np.percentile(x, p))

    return {
        "iterations": iterations,
        "duration_days": {"p50": pct(project_duration_days,50), "p80": pct(project_duration_days,80), "mean": float(np.mean(project_duration_days))},
        "effort_person_days": {"p50": pct(total_effort_days,50), "p80": pct(total_effort_days,80), "mean": float(np.mean(total_effort_days))},
        "cost_krw": {"p50": pct(total_cost,50), "p80": pct(total_cost,80), "mean": float(np.mean(total_cost))},
    }

def _subset_plan_until_validation_gate(plan: Dict[str, Any]) -> Dict[str, Any]:
    wbs = (plan or {}).get("wbs", {}) or {}
    milestones = wbs.get("milestones", []) or []
    wps = wbs.get("work_packages", []) or []

    gate_id = None
    for ms in milestones:
        if ms.get("is_validation_gate") is True:
            gate_id = ms.get("id")
            break
    if not gate_id:
        return plan

    # keep milestones up to gate
    ms_ids = [ms.get("id") for ms in milestones if ms.get("id")]
    try:
        gate_idx = ms_ids.index(gate_id)
        keep_ms = set(ms_ids[:gate_idx+1])
    except ValueError:
        keep_ms = set(ms_ids)

    keep_wps = [wp for wp in wps if wp.get("milestone_id") in keep_ms]
    # also include dependencies closure
    wp_map = {wp.get("id"): wp for wp in wps if wp.get("id")}
    keep_set = set([wp.get("id") for wp in keep_wps if wp.get("id")])
    changed = True
    while changed:
        changed = False
        for wid in list(keep_set):
            wp = wp_map.get(wid, {})
            for d in (wp.get("dependencies", []) or []):
                if d and d in wp_map and d not in keep_set:
                    keep_set.add(d)
                    changed = True
    keep_wps = [wp_map[wid] for wid in keep_set if wid in wp_map]

    new_plan = dict(plan)
    new_wbs = dict(wbs)
    new_wbs["work_packages"] = keep_wps
    new_wbs["milestones"] = [ms for ms in milestones if ms.get("id") in keep_ms]
    new_plan["wbs"] = new_wbs
    return new_plan

def cost_sim_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    payload = {
        "idea_schema": state.get("idea_schema", {}),
        "market_model": state.get("market_model", {}),
    }
    messages = [{"role":"user","content": json.dumps(payload, ensure_ascii=False)}]
    out = run_agent(messages, cost_plan_agent)
    plan = safe_json_loads(out)

    # main simulation
    sim = _simulate_cost_plan(plan, iterations=5000, seed=42)

    # min-to-validate simulation
    sub_plan = _subset_plan_until_validation_gate(plan)
    sim_min = _simulate_cost_plan(sub_plan, iterations=3000, seed=43)

    cost_sim = {
        "mvp_scope": plan.get("mvp_scope", {}),
        "wbs": plan.get("wbs", {}),
        "validation_plan": plan.get("validation_plan", {}),
        "cost_model_assumptions": plan.get("cost_model_assumptions", {}),
        "simulation": sim,
        "min_to_validate": {"plan_hint": "until validation gate milestone", "simulation": sim_min},
        "assumptions_and_unknowns": plan.get("assumptions_and_unknowns", []),
    }

    obj = CostSimModel(**cost_sim)
    state["cost_sim"] = obj.model_dump()

    logs.append({"node":"cost_sim", "t": time.time()-t0})
    state["logs"] = logs
    return state

In [45]:
ANALYSIS_PROMPT = """
역할: evidence_pack을 기반으로 시장/경쟁/고객/리스크 신호를 요약하고,
설명가능한 점수(0~5)를 만든다.
출력(JSON only):
{
  "market": "...",
  "competition": "...",
  "customer": "...",
  "risks": "...",
  "score_explainable": {"market":3.0,"competition":2.5,"customer":3.5,"risks":2.0}
}
규칙:
- fact / interpretation / hypothesis를 문장 앞 라벨로 구분해라.
- 수치(성장률 등)는 evidence_pack에 근거가 없으면 생성하지 마라.
"""

analysis_agent = Agent(name="Analysis", instructions=ANALYSIS_PROMPT, tools=[])

def analysis_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    payload = {
        "idea_schema": state["idea_schema"],
        "evidence_pack": state.get("evidence_pack", []),
        "market_plan": state.get("market_plan", {}),
        "market_model": state.get("market_model", {}),
        "cost_sim": state.get("cost_sim", {})
    }
    messages = [{"role":"user","content": json.dumps(payload, ensure_ascii=False)}]
    out = run_agent(messages, analysis_agent)
    data = safe_json_loads(out)

    obj = SignalsModel(**data)
    state["signals"] = obj.model_dump()

    logs.append({"node":"analysis", "t": time.time()-t0, "score": obj.score_explainable})
    state["logs"] = logs
    return state


In [46]:
DECISION_PROMPT = """
역할: signals + market_model + cost_sim + evidence_pack을 바탕으로
GO / NO_GO / PIVOT 결론을 내리고, 근거 링크와 다음 액션(검증 실험 포함)을 제안한다.
출력(JSON only):
{
  "decision":"GO|NO_GO|PIVOT",
  "key_reasons":["..."],
  "evidence_links":["..."],
  "next_actions":["..."]
}
규칙:
- evidence_links는 evidence_pack.meta.url에서만 가져와라(최소 3개).
- 이유(key_reasons)에 market_model(시장규모/TAM-SAM-SOM, CAGR, 생태계/경쟁구도) 요소를 최소 1개 포함.
- 이유 또는 액션(next_actions)에 cost_sim(일정/비용 p50·p80, min_to_validate 범위) 요소를 최소 1개 포함.
- 단정 금지: 불확실하면 PIVOT 또는 조건부 GO로 표현.
"""

decision_agent = Agent(name="Decision", instructions=DECISION_PROMPT, tools=[])

def decision_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    payload = {
        "idea_schema": state["idea_schema"],
        "signals": state.get("signals", {}),
        "evidence_pack": state.get("evidence_pack", []),
        "market_plan": state.get("market_plan", {}),
        "market_model": state.get("market_model", {}),
        "cost_sim": state.get("cost_sim", {})
    }
    messages = [{"role":"user","content": json.dumps(payload, ensure_ascii=False)}]
    out = run_agent(messages, decision_agent)
    data = safe_json_loads(out)

    obj = VerdictModel(**data)
    state["verdict"] = obj.model_dump()

    logs.append({"node":"decision", "t": time.time()-t0, "decision": obj.decision})
    state["logs"] = logs
    return state

def route_after_decision(state: WorkflowState) -> str:
    return "blueprint"


In [47]:
BLUEPRINT_PROMPT = '''
역할: verdict를 반영하여 MVP 설계 산출물을 만든다.
출력(JSON only):
{
  "prd_1p": "<markdown string>",
  "scope_must_should_could": "<markdown string>",
  "erd_mermaid": "```mermaid ...```",
  "user_flow": "<markdown string>",
  "roadmap_2_4_weeks": "<markdown string>",
  "validation_plan": "<markdown string>"
}
규칙:
- 위 6개 필드는 '문자열'이어야 한다. (객체/리스트 JSON으로 내지 말 것)
- scope/roadmap/validation은 사람이 읽기 좋은 bullet markdown으로 작성.
- ERD는 Mermaid ER diagram 또는 flowchart 형식.
- 검증 플랜은 '실험-지표-판정기준'이 포함되어야 함.
'''
blueprint_agent = Agent(
    name="BlueprintMaker",
    instructions=BLUEPRINT_PROMPT,
    tools=[],
)

def _to_markdown(x: Any, indent: int = 0) -> str:
    pad = "  " * indent
    if x is None:
        return ""
    if isinstance(x, str):
        return x.strip()
    if isinstance(x, list):
        lines = []
        for item in x:
            if isinstance(item, (dict, list)):
                lines.append(f"{pad}-")
                child = _to_markdown(item, indent+1)
                if child:
                    lines.append(child)
            else:
                lines.append(f"{pad}- {str(item)}")
        return "\n".join(lines).strip()
    if isinstance(x, dict):
        lines = []
        for k, v in x.items():
            if isinstance(v, (dict, list)):
                lines.append(f"{pad}- **{k}**")
                child = _to_markdown(v, indent+1)
                if child:
                    lines.append(child)
            else:
                lines.append(f"{pad}- **{k}**: {str(v)}")
        return "\n".join(lines).strip()
    return str(x).strip()

def blueprint_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    payload = {
        "idea_schema": state["idea_schema"],
        "signals": state.get("signals", {}),
        "verdict": state.get("verdict", {})
    }
    messages = [{"role":"user","content": json.dumps(payload, ensure_ascii=False)}]
    out = run_agent(messages, blueprint_agent)
    data = safe_json_loads(out)

    for key in ["prd_1p", "scope_must_should_could", "erd_mermaid", "user_flow", "roadmap_2_4_weeks", "validation_plan"]:
        if key in data and not isinstance(data[key], str):
            data[key] = _to_markdown(data[key])

    obj = ArtifactsModel(**data)
    state["artifacts"] = obj.model_dump()

    logs.append({"node":"blueprint", "t": time.time()-t0})
    state["logs"] = logs
    return state


In [48]:
GUARDRAIL_PROMPT = """
역할: 최종 산출물(analysis/verdict/artifacts)이 아래 가드를 충족하는지 점검하고,
위반/부족 플래그를 설정하며, 필요한 최소 수정(라벨/단정 표현 완화/링크 누락 보완)을 제안한다.
출력(JSON only):
{
  "policy_violation": false,
  "token_overflow": false,
  "copyright_risk": false,
  "evidence_insufficient": false,
  "notes": ["..."]
}
체크리스트:
- 핵심 주장에 출처 링크가 최소 3개 이상인가?
- fact/interpretation/hypothesis 라벨이 존재하는가?
- 수치가 '근거 없이' 생성되지 않았는가?
- 뉴스/리포트 전문을 길게 인용하지 않았는가?
"""

guardrail_agent = Agent(name="GuardrailValidator", instructions=GUARDRAIL_PROMPT, tools=[])

def guardrail_node(state: WorkflowState) -> WorkflowState:
    logs = state.get("logs", [])
    t0 = time.time()

    payload = {
        "signals": state.get("signals", {}),
        "verdict": state.get("verdict", {}),
        "artifacts": state.get("artifacts", {}),
        "evidence_pack": state.get("evidence_pack", [])
    }
    messages = [{"role":"user","content": json.dumps(payload, ensure_ascii=False)}]
    out = run_agent(messages, guardrail_agent)
    data = safe_json_loads(out)

    obj = GuardsModel(**data)
    state["guards"] = obj.model_dump()

    logs.append({"node":"guardrail", "t": time.time()-t0, "flags": {
        "policy_violation": obj.policy_violation,
        "token_overflow": obj.token_overflow,
        "copyright_risk": obj.copyright_risk,
        "evidence_insufficient": obj.evidence_insufficient
    }})
    state["logs"] = logs
    return state


In [49]:
def render_report(state: WorkflowState) -> WorkflowState:
    idea = state.get("idea_schema", {}) or {}
    signals = state.get("signals", {}) or {}
    verdict = state.get("verdict", {}) or {}
    artifacts = state.get("artifacts", {}) or {}
    guards = state.get("guards", {}) or {}
    market_model = state.get("market_model", {}) or {}
    cost_sim = state.get("cost_sim", {}) or {}
    mp = state.get("market_plan", {}) or {}

    def _money(v):
        try:
            v = float(v)
        except Exception:
            return str(v)
        if v >= 1e12:
            return f"{v/1e12:.2f}조원"
        if v >= 1e8:
            return f"{v/1e8:.2f}억원"
        if v >= 1e4:
            return f"{v/1e4:.1f}만원"
        return f"{v:.0f}원"

    # Evidence links (unique, order-preserving)
    links = []
    for ep in (state.get("evidence_pack", []) or []):
        for ch in (ep.get("top_chunks", []) or []):
            url = ((ch.get("meta", {}) or {}).get("url","") or "").strip()
            if url and url not in links:
                links.append(url)
    # also include verdict links
    for u in (verdict.get("evidence_links", []) or []):
        if u and u not in links:
            links.append(u)

    out = []
    out.append("# IdeaProof 리포트 (KR)\n")

    # --- Problem ---
    out.append("## Problem\n")
    out.append(f"- **문제(요약):** {idea.get('problem','')}\n")
    out.append(f"- **타겟(가정):** {idea.get('target','')}\n")
    if signals:
        if signals.get("market"): out.append(f"- **시장 신호:** {signals.get('market')}\n")
        if signals.get("customer"): out.append(f"- **고객 신호:** {signals.get('customer')}\n")
        if signals.get("risks"): out.append(f"- **리스크 신호:** {signals.get('risks')}\n")

    # --- Solution ---
    out.append("\n## Solution\n")
    out.append(f"- **해결책(요약):** {idea.get('solution','')}\n")
    out.append(f"- **차별점:** {idea.get('differentiation','')}\n")
    out.append(f"- **BM(수익모델):** {idea.get('business_model','')}\n")
    if artifacts:
        if artifacts.get("mvp_scope"):
            out.append("\n### MVP Scope\n")
            out.append(artifacts.get("mvp_scope","")+"\n")
        if artifacts.get("user_flow"):
            out.append("\n### User Flow\n")
            out.append(artifacts.get("user_flow","")+"\n")
        if artifacts.get("architecture"):
            out.append("\n### Architecture\n")
            out.append(artifacts.get("architecture","")+"\n")

    # --- Scale-up ---
    out.append("\n## Scale-up\n")

    # C: market model
    out.append("\n### C) 거시→미시→생태계 분석 (대한민국)\n")
    if market_model.get("market_scope"):
        out.append(f"- **시장 범위:** {market_model.get('market_scope')}\n")
    tss = market_model.get("tam_sam_som", {}) or {}
    if tss:
        out.append("- **TAM/SAM/SOM(범위):**\n")
        for k,label in [("tam_krw_range","TAM"),("sam_krw_range","SAM"),("som_krw_range","SOM")]:
            r = tss.get(k, {}) or {}
            if r:
                out.append(f"  - {label}: {_money(r.get('min',0))} ~ {_money(r.get('max',0))} (base {_money(r.get('base',0))}, {r.get('basis','')})\n")
    cagr = market_model.get("cagr", {}) or {}
    if cagr.get("scenario_pct"):
        sc = cagr.get("scenario_pct", {}) or {}
        out.append(f"- **CAGR 시나리오(%):** 보수 {sc.get('conservative')} / 기준 {sc.get('base')} / 공격 {sc.get('aggressive')} ({sc.get('basis','')})\n")
    eco = market_model.get("ecosystem", {}) or {}
    firms = eco.get("firms", []) or []
    if firms:
        out.append(f"- **생태계 기업(최대 25):** {', '.join([f.get('name','') for f in firms if f.get('name')][:25])}\n")
    if eco.get("distribution_summary"):
        out.append(f"- **분포 요약:** {eco.get('distribution_summary')}\n")
    if market_model.get("risks"):
        out.append("- **Scale-up 리스크:**\n")
        for r in market_model.get("risks", [])[:8]:
            out.append(f"  - {r}\n")

    # B: cost & schedule simulation
    out.append("\n### B) 최소비용 MVP 개발계획 + 시뮬레이션(가정)\n")
    sim = (cost_sim.get("simulation", {}) or {})
    if sim:
        dur = sim.get("duration_days", {}) or {}
        cost = sim.get("cost_krw", {}) or {}
        eff = sim.get("effort_person_days", {}) or {}
        out.append(f"- **예상 일정(의존성 반영):** p50 {dur.get('p50')}일 / p80 {dur.get('p80')}일\n")
        out.append(f"- **예상 노력(총 person-day):** p50 {eff.get('p50')} / p80 {eff.get('p80')}\n")
        out.append(f"- **예상 비용(총):** p50 {_money(cost.get('p50',0))} / p80 {_money(cost.get('p80',0))}\n")
    minv = (cost_sim.get("min_to_validate", {}) or {}).get("simulation", {}) or {}
    if minv:
        dur = minv.get("duration_days", {}) or {}
        cost = minv.get("cost_krw", {}) or {}
        out.append(f"- **검증 완료(게이트)까지 최소 범위:** p50 {dur.get('p50')}일, p80 {dur.get('p80')}일 / 비용 p50 {_money(cost.get('p50',0))}\n")
    wbs = cost_sim.get("wbs", {}) or {}
    if wbs.get("milestones"):
        out.append("\n#### Milestones\n")
        for ms in wbs.get("milestones", [])[:10]:
            gate = " (VALIDATION GATE)" if ms.get("is_validation_gate") else ""
            out.append(f"- {ms.get('id')}: {ms.get('name')}{gate}\n")
    if wbs.get("work_packages"):
        out.append("\n#### Work Packages (Top)\n")
        for wp in wbs.get("work_packages", [])[:12]:
            e = wp.get("effort_days_pert", {}) or {}
            out.append(f"- {wp.get('id')}: {wp.get('name')} [{wp.get('stage')}] (PERT {e.get('optimistic')}/{e.get('most_likely')}/{e.get('pessimistic')}d)\n")

    # Decision inside Scale-up
    out.append("\n### 판정 (GO / NO_GO / PIVOT)\n")
    out.append(f"- **결론:** {verdict.get('decision','')}\n")
    if verdict.get("key_reasons"):
        out.append("- **핵심 근거:**\n")
        for r in verdict.get("key_reasons", [])[:8]:
            out.append(f"  - {r}\n")
    if verdict.get("next_actions"):
        out.append("- **다음 액션:**\n")
        for a in verdict.get("next_actions", [])[:10]:
            out.append(f"  - {a}\n")

    # Evidence and guardrail as subsections (still within Scale-up)
    if links:
        out.append("\n### 근거 링크(Top)\n")
        for u in links[:12]:
            out.append(f"- {u}\n")

    if guards:
        out.append("\n### Guardrail\n")
        out.append("```json\n"+json.dumps(guards, ensure_ascii=False, indent=2)+"\n```\n")

    # logs
    logs = state.get("logs", [])
    if logs:
        out.append("\n### Logs (Tail)\n")
        out.append("```json\n"+json.dumps(logs[-20:], ensure_ascii=False, indent=2)+"\n```\n")

    state["final_report_markdown"] = "\n".join(out)
    return state

In [50]:
from langgraph.graph import StateGraph, START, END

graph = StateGraph(WorkflowState)

# --- nodes ---
graph.add_node("intake", intake_node)
graph.add_node("structurer", structurer_node)

# Group 2 (C + B)
graph.add_node("market_plan", market_plan_node)
graph.add_node("evidence_builder", evidence_builder_node)
graph.add_node("extractor", extractor_node)
graph.add_node("market_model", market_model_node)
graph.add_node("cost_sim", cost_sim_node)

graph.add_node("analysis", analysis_node)
graph.add_node("decision", decision_node)
graph.add_node("blueprint", blueprint_node)
graph.add_node("guardrail", guardrail_node)
graph.add_node("render", render_report)

# --- edges ---
graph.add_edge(START, "intake")
graph.add_conditional_edges(
    "intake",
    route_after_intake,
    {
        "chat_end": END,
        "structurer": "structurer",
    },
)

graph.add_edge("structurer", "market_plan")
graph.add_edge("market_plan", "evidence_builder")
graph.add_edge("evidence_builder", "extractor")
graph.add_edge("extractor", "market_model")
graph.add_edge("market_model", "cost_sim")
graph.add_edge("cost_sim", "analysis")
graph.add_edge("analysis", "decision")

# decision -> blueprint (always for now, but keep as conditional hook)
graph.add_conditional_edges("decision", route_after_decision, {"blueprint": "blueprint"})
graph.add_edge("blueprint", "guardrail")
graph.add_edge("guardrail", "render")
graph.add_edge("render", END)

app = graph.compile()


In [51]:
# 아이디어 입력: "기획서 자체"를 아이디어로 사용 (사용자 요구사항 반영)
IDEA_TEXT = """
IdeaProof(가칭)는 창업 아이디어를 입력하면 산업 분류 → 시장/경쟁/고객 신호 수집 → 근거 기반 검증 →
Go/No-Go/Pivot 결론을 제공하고, 결론에 따라 피벗 제안과 MVP 설계 산출물(ERD, 로드맵, 검증 플랜)을
패키지로 생성하는 서비스이다.
""".strip()

state0: WorkflowState = {
    "request": {"raw_request": IDEA_TEXT, "language": "ko", "tone": "concise", "mode": "standard"},
    "logs": []
}

# 예: intake + structurer만 단독 테스트
tmp = intake_node(state0.copy())
tmp = structurer_node(tmp)
rprint(tmp["idea_schema"])
rprint(tmp["logs"])


{
    'problem': '창업 아이디어 검증 과정의 비체계성, 시장 신호 수집의 어려움, MVP 설계의 전문성 부족으로 인한 실패 
리스크 증가',
    'target': '초기 창업자, 스타트업 팀, 예비 창업자, 벤처 캐피탈의 초기 단계 투자 심사 담당자',
    'solution': 'AI 기반 창업 아이디어 검증 플랫폼으로, 산업 분류 → 데이터 기반 시장/경쟁/고객 분석 → 객관적 검증 
프로세스 → 실행 가능한 피벗 제안 및 MVP 설계 산출물 패키지 제공',
    'differentiation': '1) 자동화된 산업 분류 및 신호 수집 시스템 2) 검증된 프레임워크 기반의 의사결정 지원 3) MVP 
설계(ERD/로드맵 등)의 즉시 생성 기능 4) 피벗 제안의 데이터 기반 접근',
    'business_model': 'SaaS 구독 모델(Basic/Pro/Enterprise) + 프리미엄 검증 리포트 유료 판매 + 액셀러레이터 
파트너십 수수료',
    'industry': '스타트업 인큐베이팅/액셀러레이팅, SaaS, AI 기반 의사결정 지원 도구',
    'keywords': [
        '창업 검증',
        '스타트업 MVP 설계',
        '시장 신호 분석',
        'AI 기반 의사결정',
        '피벗 전략',
        'ERD 생성',
        '로드맵 자동화',
        '창업 실패 리스크 감소'
    ],
    'persona_hypotheses': [
        '초기 창업자는 검증 과정의 전문성 부족으로 인해 서비스 의존도가 높을 것',
        '액셀러레이터는 포트폴리오 회사들의 검증 효율화를 위해 기업용 플랜에 관심을 가질 것'
    ]
}

[
    {'node': 'intake', 't': 3.7074832916259766, 'route': 'workflow'},
    {
        'node': 'structurer',
        't': 2.9190919399261475,
        'industry': '스타트업 인큐베이팅/액셀러레이팅, SaaS, AI 기반 의사결정 지원 도구'
    }
]

In [52]:
# 실행 + 진행상황(노드별) 출력: 어디서 오래 걸리는지 바로 보이게 함
state_init: WorkflowState = {"request": {"raw_request": IDEA_TEXT, "language":"ko", "tone":"concise", "mode":"standard"}, "logs":[]}

last_log_len = 0
final_state = None

for s in app.stream(state_init, stream_mode="values"):
    logs = s.get("logs", [])
    if len(logs) > last_log_len:
        for item in logs[last_log_len:]:
            print("[LOG]", item)
        last_log_len = len(logs)
    final_state = s

print("\n\n================ FINAL REPORT (markdown) ================\n")
print((final_state or {}).get("final_report_markdown","")[:4000])

[LOG] {'node': 'intake', 't': 2.864471912384033, 'route': 'workflow'}
[LOG] {'node': 'structurer', 't': 3.85493540763855, 'industry': 'AI SaaS, 스타트업 지원 서비스, 비즈니스 인텔리전스, 창업 교육 플랫폼'}
[LOG] {'node': 'market_plan', 't': 14.844451665878296, 'layers': 4}
[LOG] {'node': 'evidence_builder', 't': 237.807373046875, 'cache': 'MISS', 'collection': 'ideaproof_e224f797096a9bd7', 'mode': 'standard', 'sources': 50, 'stored_chunks': 67, 'time_budget_sec': 720, 'time_budget_hit': False}
[LOG] {'node': 'extractor', 't': 29.479384899139404, 'questions': 4}
[LOG] {'node': 'market_model', 't': 13.484107971191406}
[LOG] {'node': 'cost_sim', 't': 18.11776614189148}
[LOG] {'node': 'analysis', 't': 4.254802465438843, 'score': {'market': 3.0, 'competition': 2.5, 'customer': 3.5, 'risks': 2.0}}
[LOG] {'node': 'decision', 't': 7.119373798370361, 'decision': 'PIVOT'}
[LOG] {'node': 'blueprint', 't': 8.88239312171936}
[LOG] {'node': 'guardrail', 't': 3.179896593093872, 'flags': {'policy_violation': False, 'token_ove

In [53]:
out_path = "ideaproof_report.md"
with open(out_path, "w", encoding="utf-8") as f:
    f.write(final_state.get("final_report_markdown",""))
print("saved:", out_path)


saved: ideaproof_report.md
